In [1]:
import langchain 

In [5]:
from  langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import pinecone

In [6]:
# pinecone vector Data base API 
PINECONE_API_KEY = '617b3621-e5eb-4f98-9720-ce4c852e0faa'
PINECONE_API_ENV = 'Serverless'

In [7]:
# load Data from pdf 

def load_pdf(data):
   loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
   doc = loader.load()

   return doc

In [8]:
ExtractData =load_pdf("Media/workflow planing.png")

ValueError: Expected directory, got file: 'Media/workflow planing.png'